In [1]:
import math
import concurrent.futures
import numpy as np
import cupy as cp

In [2]:
def eye(n):
  lst = [[] for _ in range(n)]
  for i in range(n):
    for j in range(n):
      if i == j:
        lst[i].append(1)
      else:
        lst[i].append(0)
  return cp.array(lst)


In [3]:
def qr_algorithm(A):
  if isinstance(A, np.ndarray):
    A = cp.array(A)
  n = A.shape[0]
  Q = eye(n)

  for i in range(n - 1):
    R = cp.zeros((n, n))
    Q_i = cp.zeros((n, n))

    for j in range(n):
      R[j, j] = cp.sqrt(cp.sum(A[:, j] ** 2))
      Q_i[:, j] = A[:, j] / R[j, j]
      R[j, i] = cp.sum(Q_i[:, j] * A[:, i])
      A[:, i] = A[:, i] - R[j, i] * Q_i[:, j]
    
    A = R @ Q_i
    Q = Q @ Q_i
  
  eigenvalues = cp.diagonal(A)
    
  eigenvectors = Q
  
  return eigenvalues, eigenvectors

In [4]:
def compute_AtA_ij(i, j, m):
  result = 0
  for k in range(m):
    result += A[k][i]*A[k][j]
  return i, j, result

In [5]:
def svd(A):
  # Converting A to a CuPy array if it is a NumPy array
  if isinstance(A, np.ndarray):
    A = cp.array(A)
  m, n = len(A), len(A[0])
  AtA = cp.zeros((n, n))
  
  # Useing the ProcessPoolExecutor to parallelize the for loop
  with concurrent.futures.ProcessPoolExecutor() as executor:
    for i, j, result in executor.map(compute_AtA_ij, range(n), range(n), [m]*n*n, chunksize=2):
      AtA[i][j] = result
  
  eigenvalues, eigenvectors = qr_algorithm(AtA)
  singular_values = [math.sqrt(eigenvalue) for eigenvalue in eigenvalues]
  U = cp.zeros((n, m))
  for i in range(n):
    for j in range(m):
      U[i][j] = A[j][i]/singular_values[i]
  V = np.zeros((n, n))
  for i in range(n):
    for j in range(n):
      V[i][j] = eigenvectors[i][j]
  V = cp.dot(U, cp.array(V))
  S = cp.array(singular_values)
  S_diag = cp.diag(S)
  return U, S_diag, V

In [6]:
# Generating a random matrix
A = np.random.rand(4, 4)
# r = int(input("Enter the number of rows: "))
# c = int(input("Enter the number of columns: "))

# # if r != c:
# #   raise ValueError('If the matrix M is not square, then it cannot be factorized using traditional matrix factorization techniques such as SVD or QR decomposition.')

# A = [[int(input('Please Input the number for position ({},{}): '.format(x, y))) for x in range(c)] for y in range(r)]
# M = np.array(A).reshape(r, c)

U, S, Vt = svd(A) 
# Calculate M1 and M2 from the SVD
M1 = cp.dot(U, cp.diag(S))
M2 = Vt
print('M1 Value is: ',M1)
print('M2 Value is: ',M2)

M1 Value is:  [2.08164054 2.04773417 2.57252242 2.45825321]
M2 Value is:  [[0.65516238 0.09468172 0.73196849 0.16130638]
 [0.63405243 0.23581915 0.73595182 0.02723545]
 [0.53528836 0.38229936 0.57513086 0.48635179]
 [0.16737659 0.4496117  0.61230722 0.62842204]]
